In [1]:
from langchain_openai import ChatOpenAI
from crewai_tools import PDFSearchTool
from langchain_community.tools.tavily_search import TavilySearchResults
from crewai_tools import tool
from crewai import Crew
from crewai import Task
from crewai import Agent


/home/pro/miniconda3/envs/agntrag/lib/python3.10/site-packages/crewai/telemetry/telemetry.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/home/pro/miniconda3/envs/agntrag/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:937: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warnings.warn(


In [2]:
import os

os.environ["GROQ_API_KEY"] = ""

In [ ]:
llm = ChatOpenAI(
openai_api_base="https://api.groq.com/openai/v1",
openai_api_key=os.environ [GROQ APT KEY],
model_name "1lama3-86-8192",
temperature-0.1,
max tokens 1000,
)

In [ ]:
import requests
pdf_url = "https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf"
response requests.get(pdf_url)
with open("attenstion_is_all_you_need.pdf", "wb") as file:
file.write(response.content)

In [ ]:
# file: rag_demo.py
from crewai_tools import PDFSearchTool

# -------------------------------------------------
# 1. Build the tool
# -------------------------------------------------
rag_tool = PDFSearchTool(
    # pdf="attention_is_all_you_need.pdf",   # <-- make sure the file exists in cwd
    config=dict(
        # ---------- LLM (Groq) ----------
        llm=dict(
            provider="groq",               # exact string required by crewai-tools
            # api_key=None,                  # reads from env var GROQ_API_KEY
            config=dict(
                model="llama3-8b-8192",    # any Groq model name
                # temperature=0.5,        # uncomment if you want
                # top_p=1,
                # stream=True,
            ),
        ),
        # ---------- Embedder (HuggingFace) ----------
        embedder=dict(
            provider="huggingface",        # exact string
            config=dict(
                model="BAAI/bge-small-en-v1.5",
                # task_type="retrieval_document",   # optional, default is fine
                # title="Embeddings",
            ),
        ),
    ),
)

# -------------------------------------------------
# 2. Quick test – ask a question about the PDF
# -------------------------------------------------
# question = "What is the main contribution of the Transformer model?"
# answer = rag_tool.run(question)

# print("\n--- ANSWER ---")
# print(answer)

In [ ]:
rag_tool.run("What is the main contribution of the Transformer model?")

In [ ]:
import os

os.environ["TAVILY_API_KEY"] = ""

In [ ]:
web_search_tool = TavilySearchResults(k=3)

In [ ]:
web_search_tool.run("what is self attention mechansim in large languges model")

In [ ]:
# --------------------------------------------------------------
# 3. Web-search tool (DuckDuckGo – no API key needed)
# --------------------------------------------------------------
# web_search_tool = DuckDuckGoSearchRun()

@tool
def router_tool(question):
"""Router Function"""
if 'self attention' in question:
    return 'vectorstore'
else:
    return 'web search'

# @tool
# def router_tool(question: str) -> str:
#     """Router that decides between vectorstore (PDF) and web search."""
#     # Simple keyword logic – you can replace with an LLM call later
#     if "self attention" in question.lower() or "transformer" in question.lower():
#         return "vectorstore"
#     else:
#         return "web_search"

In [ ]:
Router_Agent = Agent(
    role='Router',
    goal='Route user question to a vectorstore or web search',
    backstory=(
        "You are an expert at routing a user question to a vectorstore or web search. "
        "Use the vectorstore for questions related to Retrieval-Augmented Generation (RAG), "
        "vector databases, embeddings, or the 'Attention Is All You Need' paper. "
        "You do not need to be stringent with the keywords in the question related to these topics."
    ),
    verbose True,
    allow_delegation=False,
    llm=llm,
)

# Router_Agent = Agent(
#     role='Router',
#     goal='Route user question to a vectorstore or web search',
#     backstory=(
#         "You are an expert at routing a user question to a vectorstore or web search. "
#         "Use the vectorstore for questions related to Retrieval-Augmented Generation (RAG), "
#         "vector databases, embeddings, or the 'Attention Is All You Need' paper. "
#         "You do not need to be stringent with the keywords in the question related to these topics."
#     ),
#     verbose=True,
#     allow_delegation=False,
#     llm="groq/llama3-8b-8192",  # Fixed: was "11m-11m" → correct Groq model
# )

In [ ]:
Retriever_Agent=Agent(
role="Retriever",
goal="Use the information retrieved from the vectorstore to answer the question",
backstory=(

"You are an assistant for question-answering tasks."
"Use the information present in the retrieved context to answer the question."
"You have to provide a clear concise answer."
),
verbose=True,
allow_delegation=False,
llm=llm,
)

'''Use ChatGroq instance (recommended for control)'''
# from langchain_groq import ChatGroq
# llm = ChatGroq(model="llama3-8b-8192", temperature=0.3)
# Retriever_Agent = Agent(
#     role="Retriever",
#     goal="Use the information retrieved from the vectorstore to answer the question",
#     backstory=(
#         "You are an assistant for question-answering tasks. "
#         "Use the information present in the retrieved context to answer the question. "
#         "You have to provide a clear and concise answer."
#     ),
#     verbose=True,
#     allow_delegation=False,
#     llm=llm,
# )

In [ ]:
Grader_agent=Agent(
role="Answer Grader",
goal='Filter out erroneous retrievals',
backstory=(
"You are a grader assessing relevance of a retrieved document to a user question."
"If the document contains keywords related to the user question, grade it as relevant."
"It does not need to be a stringent test. You have to make sure that the answer is relevant to the question."
)
verbose=True,
allow=delegation=False,
llm=llm,
)

'''Use ChatGroq for more control'''
# from langchain_groq import ChatGroq
# llm = ChatGroq(model="llama3-8b-8192", temperature=0.1)
# Grader_agent = Agent(
#     role="Answer Grader",
#     goal="Filter out erroneous retrievals",
#     backstory=(
#         "You are a grader assessing the relevance of a retrieved document to a user question. "
#         "If the document contains keywords related to the user question, grade it as relevant. "
#         "It does not need to be a stringent test. You must ensure that the answer is relevant to the question."
#     ),
#     verbose=True,
#     allow_delegation=False,
#     llm=llm,
# )

In [ ]:
hallucination_grader=Agent(
role="Hallucination Grader",
goal-"Filter out hallucination",
backstory-(
"You are a hallucination grader assessing whether an answer is grounded in /supported by a set of facts."
"Make sure you meticulously review the answer and check if the response provided is in alignmnet with the question asked"
),
verbose=True,
allow_delegation=False,
ll=llm,
)
'''Enhanced version with ChatGroq (recommended for precision)'''
# from langchain_groq import ChatGroq
# llm = ChatGroq(model="llama3-8b-8192", temperature=0.0)  # Low temp for factual grading
# hallucination_grader = Agent(
#     role="Hallucination Grader",
#     goal="Filter out hallucinations",
#     backstory=(
#         "You are a hallucination grader assessing whether an answer is grounded in and supported by a set of facts. "
#         "Make sure you meticulously review the answer and check if the response provided is in alignment with the question asked. "
#         "Output 'YES' if the answer is fully grounded, 'NO' if it contains hallucinations."
#     ),
#     verbose=True,
#     allow_delegation=False,
#     llm=llm,
# )

In [ ]:
answer_grader=Agent(
role="Answer Grader",
goal="Filter out hallucination from the answer.",
backstory=(
"You are a grader assessing whether an answer is useful to resolve a question."
"Make sure you meticulously review the answer and check if it makes sense for the question asked"
"If the answer is relevant generate a clear and concise response."
"If the answer gnerated is not relevant then perform a websearch using 'web_search_tool"
)
verbose=True,
allow_delegation=False,
llm=llm,
)

'''Enhanced Version with ChatGroq + Web Search Delegation'''
# from langchain_groq import ChatGroq
# from crewai_tools import tool
# from langchain_community.tools import DuckDuckGoSearchRun

# web_search_tool = DuckDuckGoSearchRun()

# llm = ChatGroq(model="llama3-8b-8192", temperature=0.1)

# answer_grader = Agent(
#     role="Answer Grader",
#     goal="Ensure the answer is relevant and grounded; fallback to web search if needed.",
#     backstory=(
#         "You are a grader assessing whether an answer is useful to resolve a question. "
#         "Make sure you meticulously review the answer and check if it makes sense for the question asked. "
#         "If the answer is relevant, generate a clear and concise response. "
#         "If the answer is not relevant or contains hallucinations, use the 'web_search_tool' to find a better answer."
#     ),
#     verbose=True,
#     allow_delegation=False,  # Keep False if you pass tools explicitly
#     llm=llm,
#     tools=[web_search_tool],  # Give it access to search
# )


In [ ]:
router_task=Task(
description=(
    "Analyze the keywords in the question: {question}\n"
    "Based on the keywords, decide whether it is eligible for a vectorstore search or a web search.\n"
    "Return a single word: 'vectorstore' if eligible for vectorstore search.\n"
    "Return a single word: 'websearch' if eligible for web search.\n"
    "Do not provide any preamble or explanation."
),
expected_output=(
        "A single word: either 'vectorstore' or 'websearch' based on the question.\n"
        "Do not provide any preamble or explanation."
    ),
agent=Router_Agent,
tools=[router_tool],
)